In [6]:
from google.colab import files

uploaded = files.upload()

Saving jigsaw-toxic-comment-train.csv.zip to jigsaw-toxic-comment-train.csv.zip
Saving validation.csv.zip to validation.csv.zip


In [7]:
!unzip jigsaw-toxic-comment-train.csv.zip
!unzip validation.csv.zip

Archive:  jigsaw-toxic-comment-train.csv.zip
  inflating: jigsaw-toxic-comment-train.csv  
Archive:  validation.csv.zip
  inflating: validation.csv          


In [8]:
import pandas as pd
train = pd.read_csv('jigsaw-toxic-comment-train.csv')
val = pd.read_csv('validation.csv')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [9]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [10]:
val

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0
...,...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,it,0
7996,7996,El imbesil ete dela luna no se entera ni ostia...,es,1
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,tr,1
7998,7998,El mapa del reinado de Alhaken esta ligerament...,es,0


# Описательный анализ тестовой выборки в сравнение с обучающей - (какие языки представлены, как они по статистикам отличаются от трейна) - 1.75 балла


In [11]:
val['lang'].unique()

array(['es', 'it', 'tr'], dtype=object)

In [12]:
stats = {}
stats["en"] = {"non-toxic": train[train['toxic'] == 0].shape[0], "toxic": train[train['toxic'] == 1].shape[0]}
for lang in val['lang'].unique():
  stats[lang] = {"non-toxic": val[(val['lang'] == lang) & (val['toxic'] == 0)].shape[0], "toxic": val[(val['lang'] == lang) & (val['toxic'] == 1)].shape[0]}

for lang in stats:
  toxic = stats[lang]["toxic"]
  non_toxic = stats[lang]["non-toxic"]
  print(f"lang: {lang}, toxic: {toxic}, non-toxic: {non_toxic}, ratio: {round((toxic+non_toxic)/toxic)}:{round((toxic+non_toxic)/non_toxic)}")

lang: en, toxic: 21384, non-toxic: 202165, ratio: 10:1
lang: es, toxic: 422, non-toxic: 2078, ratio: 6:1
lang: it, toxic: 488, non-toxic: 2012, ratio: 5:1
lang: tr, toxic: 320, non-toxic: 2680, ratio: 9:1


Как мы видим, в val представлены данные по трем языкам - испанскому, итальянскому и турецкому; в трейн выборке у нас есть лишь английский язык.  
Данных на английском языке ~100 раз больше, чем на других языках, при этом соотношения классов разнятся - для турецкого и английского соотношение классов ~ 10:1, для итальянского и испанского ~ 6:1

In [13]:
train['text_length'] = train['comment_text'].str.len()
val['text_length'] = val['comment_text'].str.len()

In [14]:
print("train, non-toxic: \n", train[train['toxic'] == 0]['text_length'].describe(), "\n")
print("train, toxic: \n", train[train['toxic'] == 1]['text_length'].describe(), "\n")
for lang in val['lang'].unique():
  print(f"{lang}, val, non-toxic: \n", val[(val['lang'] == lang) & (val['toxic'] == 0)]['text_length'].describe(), "\n")
  print(f"{lang}, val, toxic: \n", val[(val['lang'] == lang) & (val['toxic'] == 1)]['text_length'].describe(), "\n")

train, non-toxic: 
 count    202165.000000
mean        402.691178
std         589.421496
min           1.000000
25%         100.000000
50%         215.000000
75%         450.000000
max        5000.000000
Name: text_length, dtype: float64 

train, toxic: 
 count    21384.000000
mean       280.604097
std        613.588485
min          3.000000
25%         55.000000
50%        115.000000
75%        250.000000
max       5000.000000
Name: text_length, dtype: float64 

es, val, non-toxic: 
 count    2078.000000
mean      329.911453
std       229.772217
min        55.000000
25%       165.000000
50%       257.000000
75%       419.750000
max      1559.000000
Name: text_length, dtype: float64 

es, val, toxic: 
 count     422.000000
mean      285.921801
std       189.783552
min        48.000000
25%       152.000000
50%       223.000000
75%       361.500000
max      1089.000000
Name: text_length, dtype: float64 

it, val, non-toxic: 
 count    2012.000000
mean      347.234095
std       252.975916

Как мы можем заметить, для всех языков верно утверждение, что токсичный комментарий в среднем короче, чем нетоксичный; интересно, что std в английском у токсичных комментариев больше, для других языков верно обратное - с чем это связано, непонятно.  
При этом и минимальная длина комментария в английском языке меньше у нетоксичного комментария, хотя в других языках ситуация обратная.

# Поиск признаков, которые могут обобщаться на многоязычные данные (минимум 10 признаков рассмотрено,, обучать можно любую модель, оценивать нужно на validation.csv)  - 3 балла

In [15]:
import re 
from string import punctuation

# делаем минимум нормализации т.к. хотим учитывать как можно больше признаков
def soft_normalize(text):
  text = re.sub("\n|\t|\s\"", " ", text)
  return text

def tokenize(text):
  text = re.sub(f"[{punctuation}]", " ", text)
  text = re.sub("\s+", " ", text)
  return text.split()

def sentenize(text):
  sents = re.split("[\n.!?]( |$)", text)
  return [s for s in sents if len(s) > 0 and s != " "]

def sents_count(text):
  return len(sentenize(text))

def mean_token_length(text):
  length = 0
  tokens = tokenize(text)
  if len(tokens) == 0:
    return 0
  for token in tokens:
    length += len(token)
  return length/len(tokens)

def caps_ratio(text):
  total = len(text)
  caps = 0
  for letter in text:
    if letter not in punctuation and letter != " ":
      if letter.lower() != letter:
        caps += 1
  return caps/total

def nonalpha_ratio(text):
  total = len(text)
  nonalpha = 0
  sents = sentenize(text)
  for sent in sents:
    for letter in text:
      if letter in punctuation:
        nonalpha += 1
  return nonalpha/total

def numeric_ratio(text):
  total = len(text)
  numeric = 0
  for letter in text:
    if letter.isnumeric():
      numeric += 1
  return numeric/total

def has_link(text):
  return int(re.search("https?://[^ ]+", text) is not None)

def longest_word(text):
  tokens = tokenize(text)
  length = 0
  for t in tokens:
    if len(t) > length:
      length = len(t)
  return length

def max_sent_complexity(text):
  compl = 0
  sents = sentenize(text)
  for s in sents:
    matches = re.findall("[,;\:\-()]+", s)
    if len(matches) > compl:
      compl = len(matches)
  return compl

def rage_punctuation_length(text):
  length = 0
  matches = re.findall("[?!.]+", text)
  for m in matches:
    if len(m) > length:
      length = len(m)
  return length

def longest_same_char(text):
  length = 0
  curr_length = 1
  for i in range(1,len(text)):
    if text[i] == text[i-1]:
      curr_length += 1
    else:
      if curr_length > length:
        length = curr_length
      curr_length = 1
  if curr_length > length:
    return curr_length
  return length 

In [16]:
train['text_soft_normalized'] = train['comment_text'].apply(soft_normalize)

train['text_mean_tokens_length'] = train['text_soft_normalized'].apply(mean_token_length)
train['text_sents_count'] = train['comment_text'].apply(sents_count)
train['text_caps_ratio'] = train['comment_text'].apply(caps_ratio)
train['text_nonalpha_ratio'] = train['text_soft_normalized'].apply(nonalpha_ratio)
train['text_numeric_ratio'] = train['comment_text'].apply(numeric_ratio)
train['text_has_link'] = train['comment_text'].apply(has_link)
train['text_longest_word'] = train['text_soft_normalized'].apply(longest_word)
train['text_complexity'] = train['text_soft_normalized'].apply(max_sent_complexity)
train['text_rage_punctuation_length'] = train['text_soft_normalized'].apply(rage_punctuation_length)
train['text_longest_same_char'] = train['text_soft_normalized'].apply(longest_same_char)

In [17]:
val['text_soft_normalized'] = val['comment_text'].apply(soft_normalize)

val['text_mean_tokens_length'] = val['text_soft_normalized'].apply(mean_token_length)
val['text_sents_count'] = val['comment_text'].apply(sents_count)
val['text_caps_ratio'] = val['comment_text'].apply(caps_ratio)
val['text_nonalpha_ratio'] = val['text_soft_normalized'].apply(nonalpha_ratio)
val['text_numeric_ratio'] = val['comment_text'].apply(numeric_ratio)
val['text_has_link'] = val['comment_text'].apply(has_link)
val['text_longest_word'] = val['text_soft_normalized'].apply(longest_word)
val['text_complexity'] = val['text_soft_normalized'].apply(max_sent_complexity)
val['text_rage_punctuation_length'] = val['text_soft_normalized'].apply(rage_punctuation_length)
val['text_longest_same_char'] = val['text_soft_normalized'].apply(longest_same_char)

In [18]:
X_train = train[['text_mean_tokens_length', 'text_sents_count', 'text_caps_ratio', 'text_nonalpha_ratio', 'text_numeric_ratio', 'text_has_link', 'text_longest_word', 'text_complexity', 'text_rage_punctuation_length', 'text_longest_same_char']].values
Y_train = train['toxic']
X_val = val[['text_mean_tokens_length', 'text_sents_count', 'text_caps_ratio', 'text_nonalpha_ratio', 'text_numeric_ratio', 'text_has_link', 'text_longest_word', 'text_complexity', 'text_rage_punctuation_length', 'text_longest_same_char']].values
Y_val = val['toxic']

In [19]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

splits = 10

kf = KFold(n_splits=splits, random_state=42, shuffle=True)
score_train = []
score_train_sum = 0
for train, test in kf.split(X_train):
  X_train_train, X_train_test, Y_train_train, Y_train_test = X_train[train], X_train[test], Y_train[train], Y_train[test]
  clf = DecisionTreeClassifier(class_weight = {0: 0.91, 1:0.09}, random_state=42).fit(X_train_train, Y_train_train)
  predicted = clf.predict(X_train_test)
  score = roc_auc_score(Y_train_test, predicted)
  score_train_sum += score
  score_train.append(score)

mean_score_train = score_train_sum/splits
std = 0
for score in score_train:
  std += (score - mean_score_train) * (score - mean_score_train)
  
print(std/splits)

clf = DecisionTreeClassifier(class_weight = {0: 0.91, 1:0.09}, random_state=42).fit(X_train, Y_train)
predicted = clf.predict(X_val)
score_val = roc_auc_score(Y_val, predicted)

print(mean_score_train)
print(score_val)

1.599411843405361e-05
0.5751383882122605
0.5421227077854235


Как видим, std очень маленькое, так что переобучения нет; roc_auc_score на val не особо хуже, чем на отложенных трейнах, радуемся этому!
Похоже, что какие-то из признаков действительно рабочие

# Бейзлайн модель, которая работает на отложенной выборке (из трейна) и validation.csv (различие в точности может быть большим, но главное попробовать любыми способами его уменьшить и зафиксировать эксперименты) - 3 балла

Будем использовать ту же модель, что сделали выше, попробуем ее улучшить

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

splits = 10
kf = KFold(n_splits=splits, random_state=42, shuffle=True)

def baseline(model):
  score_train = []
  score_train_sum = 0
  for train, test in kf.split(X_train):
    X_train_train, X_train_test, Y_train_train, Y_train_test = X_train[train], X_train[test], Y_train[train], Y_train[test]
    clf = model.fit(X_train_train, Y_train_train)
    predicted = clf.predict(X_train_test)
    score = roc_auc_score(Y_train_test, predicted)
    score_train_sum += score
    score_train.append(score)

  mean_score_train = score_train_sum/splits
  std = 0
  for score in score_train:
    std += (score - mean_score_train) * (score - mean_score_train)

  clf = model.fit(X_train, Y_train)
  predicted = clf.predict(X_val)
  score_val = roc_auc_score(Y_val, predicted)

  print(f"train_score: {mean_score_train}, val_score: {score_val}")

In [21]:
first_class_weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in first_class_weights:
  print(f"weights: {i}, {1-i}")
  model = DecisionTreeClassifier(class_weight = {0: i, 1: 1-i}, random_state=42)
  baseline(model)

weights: 0.1, 0.9
train_score: 0.5750142868401349, val_score: 0.5273300428720683
weights: 0.2, 0.8
train_score: 0.5767710651888928, val_score: 0.5379772069507992
weights: 0.3, 0.7
train_score: 0.5791864874919617, val_score: 0.5363547933854522
weights: 0.4, 0.6
train_score: 0.575905152958435, val_score: 0.5427465744376794
weights: 0.5, 0.5
train_score: 0.5706613109885532, val_score: 0.5438201774927646
weights: 0.6, 0.4
train_score: 0.5713590214314728, val_score: 0.5433037912358445
weights: 0.7, 0.30000000000000004
train_score: 0.5717810966336813, val_score: 0.5413078983079342
weights: 0.8, 0.19999999999999996
train_score: 0.5741593776202095, val_score: 0.5480323281814798
weights: 0.9, 0.09999999999999998
train_score: 0.5752629939451948, val_score: 0.5457055877796592


При весах классов 0.8, 0.2 мы получаем самый высокий val_score, при этом train_score не сильно падает;  
train_score: 0.5741593776202095, val_score: 0.5480323281814798

In [29]:
samples = [1,2,3,4,5]
for sample in samples:
  print(f"samples: {sample}")
  model = DecisionTreeClassifier(class_weight = {0: 0.8, 1: 0.2}, min_samples_leaf=sample, random_state=42)
  baseline(model)

samples: 1
train_score: 0.5734202641751349, val_score: 0.5417162037203829
samples: 2
train_score: 0.5544395284523362, val_score: 0.5360053319883273
samples: 3
train_score: 0.5404313664425986, val_score: 0.520071213267524
samples: 4
train_score: 0.5339024565320598, val_score: 0.5166318406167814
samples: 5
train_score: 0.5302379119245937, val_score: 0.5122713789914857


In [30]:
splitters = ['best', 'random']
for splitter in splitters:
  print(f"splitter: {splitter}")
  model = DecisionTreeClassifier(class_weight = {0: 0.8, 1: 0.2}, min_samples_leaf=1, splitter=splitter, random_state=42)
  baseline(model)

splitter: best
train_score: 0.5734202641751349, val_score: 0.5417162037203829
splitter: random
train_score: 0.5717158287794799, val_score: 0.5469611269229383


In [32]:
criterions = ['gini', 'entropy']
for criterion in criterions:
  print(f"criterion: {criterion}")
  model = DecisionTreeClassifier(class_weight = {0: 0.8, 1: 0.2}, min_samples_leaf=1, splitter='random', criterion=criterion, random_state=42)
  baseline(model)

criterion: gini
train_score: 0.5717158287794799, val_score: 0.5469611269229383
criterion: entropy
train_score: 0.5720764380461504, val_score: 0.5251606201438677


In [34]:
features = ['auto', 'sqrt', 'log2', None]
for feature in features:
  print(f"max features: {feature}")
  model = DecisionTreeClassifier(class_weight = {0: 0.8, 1: 0.2}, min_samples_leaf=1, splitter='random', criterion='gini', random_state=42, max_features=feature)
  baseline(model)

max features: auto
train_score: 0.5685961466154824, val_score: 0.5404522582892004
max features: sqrt
train_score: 0.5685961466154824, val_score: 0.5404522582892004
max features: log2
train_score: 0.5685961466154824, val_score: 0.5404522582892004
max features: None
train_score: 0.5717158287794799, val_score: 0.5469611269229383


In [37]:
baseline_model = DecisionTreeClassifier(class_weight = {0: 0.8, 1: 0.2}, min_samples_leaf=1, splitter='random', criterion='gini', random_state=42, max_features=None)
baseline_model = baseline_model.fit(X_train, Y_train)
predicted = baseline_model.predict(X_val)
score_val = roc_auc_score(Y_val, predicted)
print(score_val)
print(baseline_model.feature_importances_)

0.5469611269229383
[0.22146641 0.06784836 0.21359617 0.18782283 0.04220467 0.00031634
 0.09445116 0.05256964 0.0350566  0.0846678 ]


Как мы видим, самыми важными параметрами оказались:  
- text_mean_tokens_length - 0.22146641
- text_caps_ratio - 0.21359617
- text_nonalpha_ratio - 0.1878228